In [1]:
# !python -m pip install mlagents==0.28.0
# !python -m pip install gym
# !cd "/Users/aditya/Documents/GitHub/game_creation_research/ml-agents/gym-unity" && pip3 install -e .

import random
from PlaceAndShootGym import *
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from GAME_4_SOLVER import *
from GAME_5_SOLVER import *


In [2]:
# GAME1 = Shoot into bucket through many different gaps for variable rewards
# GAME2 = Shoot into bucket after colliding with crate once
# GAME3 = Shoot and land onto a platform on the top right made of corner and crate
# GAME4 = shoot and touch as many objects before it touches the floor and playable if more than 6 points
# GAME5 = touch all three walls and then go into the bucket

In [3]:
try:
    env.close()
except:
    pass

SERVER_BUILD = "../Builds/MLAgent_View_21April22_server.app"
GYM_BUILD = "../Builds/Gym_View_25April22.app"

channel = EngineConfigurationChannel()
channel.set_configuration_parameters(time_scale=2, quality_level=0)
# unity_env = UnityEnvironment(
#     file_name=GYM_BUILD, seed=1, side_channels=[channel], worker_id=0)

unity_env = UnityEnvironment(side_channels=[channel])

# Start interacting with the environment.
unity_env.reset()
gym_env = UnityToGymWrapper(unity_env, allow_multiple_obs=False)
env = PlaceAndShootGym(gym_env, reward_fn=GAME_4_REWARD,
                       actionTransformer=GAME_4_TRANSFORMER,
                       announce_actions=True)


[INFO] Listening on port 5004. Start training by pressing the Play button in the Unity Editor.
[INFO] Connected to Unity environment with package version 2.2.1-exp.1 and communication version 1.5.0
[INFO] Connected new brain: PlaceAndShoot?team=0
[WARNING] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.


/opt/anaconda3/envs/introml_conda/lib/python3.8/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [4]:
env.setup(GAME_4_SETUP)

array([ 6.75      ,  1.        , -3.832676  , -0.52881503,  0.45793498,
       -0.114176  , -1.5371389 ,  1.340065  , -2.6007779 ,  3.299083  ,
        0.        , -3.9       ,  0.        , -0.        , -1.        ,
        1.        ], dtype=float32)

In [10]:
# reset pedestal

a = Action()
a.reset = True
print(Obs(env.step(a)[0]))
a.describe()

[0, 0, 0, 0, None, True]
bucket: Vector2(x=6.75, y=1.0)
corner: Vector2(x=-3.832676, y=-0.52881503)
crate: Vector2(x=0.45793498, y=-0.114176)
gear: Vector2(x=-1.5371389, y=1.340065)
triangle: Vector2(x=-2.6007779, y=3.299083)
Ball Position: Vector2(x=0.0, y=-3.9)
Ball Velocity: Vector2(x=0.0, y=-0.0)
Collided With: None
In Reset?: True


Ball was reset and no other value matters.



In [12]:
# see exactly what just happened

for each_obs in env.lastObsVec:
    print(each_obs)


bucket: Vector2(x=6.75, y=1.0)
corner: Vector2(x=-3.832676, y=-0.52881503)
crate: Vector2(x=0.45793498, y=-0.114176)
gear: Vector2(x=-1.5371389, y=1.340065)
triangle: Vector2(x=-2.6007779, y=3.299083)
Ball Position: Vector2(x=-0.37661368, y=0.9981646)
Ball Velocity: Vector2(x=4.5399137, y=10.2461)
Collided With: None
In Reset?: False

bucket: Vector2(x=6.75, y=1.0)
corner: Vector2(x=-3.832676, y=-0.52881503)
crate: Vector2(x=0.45793498, y=-0.114176)
gear: Vector2(x=-1.5371389, y=1.340065)
triangle: Vector2(x=-2.6007779, y=3.299083)
Ball Position: Vector2(x=1.4017698, y=4.199694)
Ball Velocity: Vector2(x=4.362077, y=6.001949)
Collided With: None
In Reset?: False

bucket: Vector2(x=6.75, y=1.0)
corner: Vector2(x=-3.832676, y=-0.52881503)
crate: Vector2(x=0.45793498, y=-0.114176)
gear: Vector2(x=-1.5371389, y=1.340065)
triangle: Vector2(x=-2.6007779, y=3.299083)
Ball Position: Vector2(x=2.5906036, y=2.9036694)
Ball Velocity: Vector2(x=2.7788472, y=-5.8761983)
Collided With: topWall
In Res

In [8]:
# env.velTresh = 0.005

In [11]:
# manual actioner

action = Action()
action.mouseX = -0.33
action.mouseY = 0
action.objX = 0
action.objY = 0
# action.setObject("crate")
# action.reset = True
action.describe()
env.step(action)

action = Action()
action.mouseX = -0.4
action.mouseY = -0.8
action.objX = 0
action.objY = 0
# action.setObject("crate")
# action.reset = True
action.describe()
env.step(action)



Agent put mouse at world pos (-2.228, 0.0)
No object interaction in this action

[-0.33, 0, 0, 0, None, False]

Agent put mouse at world pos (-2.7, -5.4)
No object interaction in this action

[-0.4, -0.8, 0, 0, None, False]


(array([ 6.7500000e+00,  1.0000000e+00, -3.8326759e+00, -5.2881503e-01,
         4.5793498e-01, -1.1417600e-01, -1.5371389e+00,  1.3400650e+00,
        -2.6007779e+00,  3.2990830e+00,  4.4376173e+00, -4.4395351e+00,
        -2.6095062e-04,  0.0000000e+00,  7.0000000e+00,  0.0000000e+00],
       dtype=float32),
 0.0,
 False,
 {'step': <mlagents_envs.base_env.DecisionSteps at 0x7f8a13598940>})

In [10]:
env.close()

In [11]:
# env.reset()
